In [1]:
from Data_manager.UserUtils import *
from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample
import numpy as np
URM_all = getURM_all()
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
#URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.80)

In [2]:
import pickle

with open('URM_train_validation.pkl', 'rb') as f:
    URM_train_validation = pickle.load(f)

In [3]:
import pickle

with open('URM_test.pkl', 'rb') as f:
    URM_test = pickle.load(f)

In [4]:
# SETUP EVALUATORS
from Evaluation.Evaluator import EvaluatorHoldout

#evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 2203 (17.4%) Users that have less than 1 test interactions


In [5]:
import numpy as np 
import scipy.sparse as sps
profile_length = np.ediff1d(sps.csr_matrix(URM_train_validation).indptr)

#profile_length = np.ediff1d(sps.csr_matrix(URM_train_validation).indptr)
profile_length, profile_length.shape

(array([36, 28,  3, ...,  5, 20, 56], dtype=int32), (12638,))

In [5]:
block_size = int(len(profile_length)*0.1)
block_size

1263

In [17]:
sorted_users = np.argsort(profile_length)
for group_id in range(0, 10):
    start_pos = group_id * block_size
    end_pos = min((group_id+1) * block_size, len(profile_length))
    
    users_in_group = sorted_users[start_pos:end_pos]
    
    users_in_group_p_len = profile_length[users_in_group]
    
    print("Group {}, #users in group {}, average p.len {:.2f}, median {}, min {}, max {}".format(
        group_id, 
        users_in_group.shape[0],
        users_in_group_p_len.mean(),
        np.median(users_in_group_p_len),
        users_in_group_p_len.min(),
        users_in_group_p_len.max()))
    

Group 0, #users in group 1263, average p.len 0.83, median 1.0, min 0, max 2
Group 1, #users in group 1263, average p.len 2.42, median 2.0, min 2, max 3
Group 2, #users in group 1263, average p.len 4.68, median 5.0, min 3, max 6
Group 3, #users in group 1263, average p.len 7.72, median 8.0, min 6, max 10
Group 4, #users in group 1263, average p.len 11.83, median 12.0, min 10, max 14
Group 5, #users in group 1263, average p.len 17.67, median 18.0, min 14, max 21
Group 6, #users in group 1263, average p.len 25.76, median 26.0, min 21, max 31
Group 7, #users in group 1263, average p.len 37.65, median 37.0, min 31, max 46
Group 8, #users in group 1263, average p.len 58.62, median 58.0, min 46, max 76
Group 9, #users in group 1263, average p.len 132.04, median 110.0, min 76, max 512


In [6]:
from Recommenders.Neural.MultVAERecommender import MultVAERecommender
from Hybrid import *
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
from Recommenders.FactorizationMachines.LightFMRecommender import LightFMCFRecommender
from Recommenders.NonPersonalizedRecommender import TopPop
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from Recommenders.SLIM.SLIMElasticNetRecommender import *
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_AsySVD_Cython
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender

MAP_recommender_per_group = {}

collaborative_recommender_class = {"TopPop": TopPop,
                                   "UserKNNCF": UserKNNCFRecommender,
                                   "ItemKNNCF": ItemKNNCFRecommender,
                                   "P3alpha": P3alphaRecommender,
                                   #"RP3_old": RP3betaRecommender,
                                   "RP3": RP3betaRecommender,
                                   "SLIM_ELASTIC": SLIMElasticNetRecommender,
                                   #"MULTVAE": MultVAERecommender,
                                   "EASE_R": EASE_R_Recommender,
                                   }
collaborative_recommender_class.items()

recommender_object_dict = {}

for label, recommender_class in collaborative_recommender_class.items():
    recommender_object = recommender_class(URM_train_validation)
    recommender_object_dict[label] = recommender_object
    
    
recommender_object_dict["TopPop"].fit()
#recommender_object_dict["SLIM_ELASTIC"].fit(topK=8894, l1_ratio=0.05565733019999427, alpha=0.0012979360257937668)
recommender_object_dict["SLIM_ELASTIC"].load_model("slim_models/", "slim_t_val")
#recommender_object_dict["PureSVD"].fit(num_factors=70, random_seed=42)
recommender_object_dict["P3alpha"].fit(topK=76, alpha=0.377201600381895, normalize_similarity=True)
#recommender_object_dict["RP3_old"].fit(topK=101, alpha=0.3026342852596128, beta=0.058468783118329024)
recommender_object_dict["RP3"].fit(alpha=0.20026352123406477, beta=0.15999879728761354, topK=32)
recommender_object_dict["UserKNNCF"].fit(topK=469, shrink=38, similarity='asymmetric', normalize=True,
                                       feature_weighting='TF-IDF', asymmetric_alpha=0.40077406933762383)
recommender_object_dict["ItemKNNCF"].fit(topK=31, shrink=435, similarity='tversky', normalize=True,
                                       feature_weighting='BM25', tversky_alpha=0.17113169506422393, tversky_beta=0.5684024974085575)
recommender_object_dict["EASE_R"].load_model("slim_models/", "ease_t_val")
#recommender_object_dict["EASE_R"].fit(topK=24, l2_norm=37.54323189430143)

#recommender_object_dict["MULTVAE"].fit(epochs=120, learning_rate=0.0003386225308766206, l2_reg=2.129359930047709e-06, dropout=0.09838599885053126, total_anneal_steps=525579, anneal_cap=0.4033304347929101,batch_size=256)
#recommender_object_dict["MF_BPR"].fit(sgd_mode="adagrad", num_factors=219, batch_size=32, learning_rate=0.08036400470449359,positive_reg=0.016211221706684466,negative_reg=0.00012672106970228173, epochs=260, random_seed=1234)
#recommender_object_dict["SLIM_BPR"].fit(topK=46, symmetric=True, sgd_mode="adagrad", epochs=195, lambda_i=0.00035732097169170193, lambda_j=0.004542391418975173, learning_rate=0.0002291767622519941)
#recommender_object_dict["LightFM"].load_model("result_experiments/", "LightFMCFRecommender_best_model.zip")
#recommender_object_dict["IALS"].fit()

/Users/alessandro/RecSys-2023-polimi/venv/lib/python3.11/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


TopPopRecommender: URM Detected 205 ( 1.6%) users with no interactions.
TopPopRecommender: URM Detected 109 ( 0.5%) items with no interactions.
UserKNNCFRecommender: URM Detected 205 ( 1.6%) users with no interactions.
UserKNNCFRecommender: URM Detected 109 ( 0.5%) items with no interactions.
ItemKNNCFRecommender: URM Detected 205 ( 1.6%) users with no interactions.
ItemKNNCFRecommender: URM Detected 109 ( 0.5%) items with no interactions.
P3alphaRecommender: URM Detected 205 ( 1.6%) users with no interactions.
P3alphaRecommender: URM Detected 109 ( 0.5%) items with no interactions.
RP3betaRecommender: URM Detected 205 ( 1.6%) users with no interactions.
RP3betaRecommender: URM Detected 109 ( 0.5%) items with no interactions.
SLIMElasticNetRecommender: URM Detected 205 ( 1.6%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 109 ( 0.5%) items with no interactions.
EASE_R_Recommender: URM Detected 205 ( 1.6%) users with no interactions.
EASE_R_Recommender: URM Detecte

In [7]:
recommender_object_dict["IALS"] = IALSRecommender(URM_train_validation)
recommender_object_dict["IALS"].load_model("slim_models/", "IALS_train_validation.zip")

IALSRecommender: URM Detected 205 ( 1.6%) users with no interactions.
IALSRecommender: URM Detected 109 ( 0.5%) items with no interactions.
IALSRecommender: Loading model from file 'slim_models/IALS_train_validation.zip'
IALSRecommender: Loading complete


In [8]:
from Recommenders.Neural.MultVAERecommender import MultVAERecommender_OptimizerMask
final = MultVAERecommender_OptimizerMask(URM_train_validation)
final.load_model("slim_models/", "MultVAE_train_validation_300")
recommender_object_dict["MULTVAE"] = final

MultVAERecommender: URM Detected 205 ( 1.6%) users with no interactions.
MultVAERecommender: URM Detected 109 ( 0.5%) items with no interactions.
MultVAERecommender: Loading model from file 'slim_models/MultVAE_train_validation_300'


2024-01-09 17:22:50.140796: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-01-09 17:22:50.140849: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-01-09 17:22:50.140864: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-01-09 17:22:50.140918: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-01-09 17:22:50.141064: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2024-01-09 17:22:50.259452: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-01-09 17:22:50.275014: I tensorflow/core/gra

INFO:tensorflow:Restoring parameters from slim_models/MultVAE_train_validation_300/.session/session
MultVAERecommender: Loading complete


In [9]:
from Hybrid import *


recommender_object_dict["hybrid"] = ScoresHybridRecommender(URM_train_validation, recommender_object_dict["SLIM_ELASTIC"], recommender_object_dict["RP3"], recommender_object_dict["ItemKNNCF"])
recommender_object_dict["hybrid"].fit(alpha=0.5072513954300486, beta=0.30902582824263064)

SLIMElasticNetRP3betaItemKNNCFScoresHybridRecommender: URM Detected 205 ( 1.6%) users with no interactions.
SLIMElasticNetRP3betaItemKNNCFScoresHybridRecommender: URM Detected 109 ( 0.5%) items with no interactions.


In [10]:
recommender_object_dict["hybrid_new"] = LinearHybridRecommender(URM_train_validation, [recommender_object_dict["SLIM_ELASTIC"], recommender_object_dict["RP3"], recommender_object_dict["ItemKNNCF"], recommender_object_dict["EASE_R"]])
recommender_object_dict["hybrid_new"].fit(alphas=[0.8857667100747117, 0.6379807942443021, 0.0690929184825888, 0.11953478623354052])

SLIMElasticNetRP3betaItemKNNCFEASE_R_LinearHybridRecommender: URM Detected 205 ( 1.6%) users with no interactions.
SLIMElasticNetRP3betaItemKNNCFEASE_R_LinearHybridRecommender: URM Detected 109 ( 0.5%) items with no interactions.


In [13]:
recommender_object_dict["superH"] = LinearHybridRecommender(URM_train_validation, [recommender_object_dict["SLIM_ELASTIC"], recommender_object_dict["RP3"], recommender_object_dict["KNN_H"], recommender_object_dict["EASE_R"]])
recommender_object_dict["superH"].fit(alphas=[0.8857667100747117, 0.6379807942443021, 0.0690929184825888, 0.11953478623354052])

SLIMElasticNetRP3betaUserKNNCFItemKNNCFTwoScoresHybridEASE_R_LinearHybridRecommender: URM Detected 205 ( 1.6%) users with no interactions.
SLIMElasticNetRP3betaUserKNNCFItemKNNCFTwoScoresHybridEASE_R_LinearHybridRecommender: URM Detected 109 ( 0.5%) items with no interactions.


In [12]:
recommender_object_dict["KNN_H"] = TwoScoresHybridRecommender(URM_train_validation, recommender_object_dict["UserKNNCF"], recommender_object_dict["ItemKNNCF"])
recommender_object_dict["KNN_H"].fit(alpha=0.022195783788315104)

UserKNNCFItemKNNCFTwoScoresHybridRecommender: URM Detected 205 ( 1.6%) users with no interactions.
UserKNNCFItemKNNCFTwoScoresHybridRecommender: URM Detected 109 ( 0.5%) items with no interactions.


In [14]:
recommender_object_dict["lowInt"] = ScoresHybridRecommender(URM_train_validation, recommender_object_dict["KNN_H"], recommender_object_dict["MULTVAE"], recommender_object_dict["IALS"])
recommender_object_dict["lowInt"].fit(alpha=0.7509279131476281,beta=0.028318614437090585)

UserKNNCFItemKNNCFTwoScoresHybridMultVAEIALSScoresHybridRecommender: URM Detected 205 ( 1.6%) users with no interactions.
UserKNNCFItemKNNCFTwoScoresHybridMultVAEIALSScoresHybridRecommender: URM Detected 109 ( 0.5%) items with no interactions.


In [15]:
recommender_object_dict["hybrid_hi"] = TwoScoresHybridRecommender(URM_train_validation, recommender_object_dict["SLIM_ELASTIC"], recommender_object_dict["RP3"])
recommender_object_dict["hybrid_hi"].fit(alpha=0.6201320790279279)

SLIMElasticNetRP3betaTwoScoresHybridRecommender: URM Detected 205 ( 1.6%) users with no interactions.
SLIMElasticNetRP3betaTwoScoresHybridRecommender: URM Detected 109 ( 0.5%) items with no interactions.


In [16]:
recommender_object_dict["hyb_li"] = LinearHybridRecommender(URM_train_validation, [recommender_object_dict["superH"], recommender_object_dict["MULTVAE"], recommender_object_dict["IALS"]])
recommender_object_dict["hyb_li"].fit(alphas=[0.9167461740462644, 0.0389215437427841, 0.038272767002858046])

SLIMElasticNetRP3betaUserKNNCFItemKNNCFTwoScoresHybridEASE_R_LinearHybridMultVAEIALSLinearHybridRecommender: URM Detected 205 ( 1.6%) users with no interactions.
SLIMElasticNetRP3betaUserKNNCFItemKNNCFTwoScoresHybridEASE_R_LinearHybridMultVAEIALSLinearHybridRecommender: URM Detected 109 ( 0.5%) items with no interactions.


In [17]:
recommender_object_dict["h_hi_var"] = ScoresHybridRecommender(URM_train_validation, recommender_object_dict["SLIM_ELASTIC"], recommender_object_dict["RP3"], recommender_object_dict["EASE_R"])
recommender_object_dict["h_hi_var"].fit(alpha=0.577913223086042, beta=0.31411320194876674)

SLIMElasticNetRP3betaEASE_R_ScoresHybridRecommender: URM Detected 205 ( 1.6%) users with no interactions.
SLIMElasticNetRP3betaEASE_R_ScoresHybridRecommender: URM Detected 109 ( 0.5%) items with no interactions.


In [18]:
recommender_object_dict["h_veryhi"] = LinearHybridRecommender(URM_train_validation, [recommender_object_dict["SLIM_ELASTIC"], recommender_object_dict["RP3"], recommender_object_dict["P3alpha"], recommender_object_dict["EASE_R"]])
recommender_object_dict["h_veryhi"].fit(alphas=[0.6258551295673683, 0.2599284641742562, 0.16636767655647627, 0.1309311249988266])

SLIMElasticNetRP3betaP3alphaEASE_R_LinearHybridRecommender: URM Detected 205 ( 1.6%) users with no interactions.
SLIMElasticNetRP3betaP3alphaEASE_R_LinearHybridRecommender: URM Detected 109 ( 0.5%) items with no interactions.


In [19]:
recommender_object_dict["h_veryhi"] = LinearHybridRecommender(URM_train_validation, [recommender_object_dict["SLIM_ELASTIC"], recommender_object_dict["RP3"], recommender_object_dict["P3alpha"], recommender_object_dict["EASE_R"]])
recommender_object_dict["h_veryhi"].fit(alphas=[0.6945135990698177, 0.2785785166427843, 0.19500750430888225, 0.14106908598709533])

SLIMElasticNetRP3betaP3alphaEASE_R_LinearHybridRecommender: URM Detected 205 ( 1.6%) users with no interactions.
SLIMElasticNetRP3betaP3alphaEASE_R_LinearHybridRecommender: URM Detected 109 ( 0.5%) items with no interactions.


In [37]:
# PROFILE LENGTH BASED GROUPING
from Evaluation.Evaluator import EvaluatorHoldout
import numpy as np

li_steps = 1
mi_steps = 4
hi_steps = 7


MAP_recommender_per_group = {}
cutoff = 10
profile_length_cutoffs = np.arange(2, 5, li_steps)
profile_length_cutoffs = np.append(profile_length_cutoffs, np.arange(5, 20, mi_steps))
profile_length_cutoffs = np.append(profile_length_cutoffs, np.arange(20, 110, hi_steps))
profile_length_cutoffs = np.append(profile_length_cutoffs, np.inf)# Define the cutoffs for profile length for each group
#profile_length_cutoffs = [2, 3, 4, 6, 8, 25, 80, 100, 250, 1500]

for group_id in range(len(profile_length_cutoffs)):

    # Select users whose profile length is less than the cutoff for the group and greater than or equal to the cutoff for the previous group
    if group_id == 0:
        users_in_group_flag = profile_length < profile_length_cutoffs[group_id]
    else:
        users_in_group_flag = np.logical_and(profile_length >= profile_length_cutoffs[group_id - 1],
                                             profile_length < profile_length_cutoffs[group_id])
    users_in_group = np.arange(len(profile_length))[users_in_group_flag]

    if len(users_in_group) > 0:
        users_in_group_p_len = profile_length[users_in_group]

        print("Group {}, #users in group {}, average p.len {:.2f}, median {}, min {}, max {}".format(
            group_id,
            users_in_group.shape[0],
            users_in_group_p_len.mean(),
            np.median(users_in_group_p_len),
            users_in_group_p_len.min(),
            users_in_group_p_len.max()))

        # Select users not in the current group
        users_not_in_group_flag = np.isin(np.arange(len(profile_length)), users_in_group, invert=True)
        users_not_in_group = np.arange(len(profile_length))[users_not_in_group_flag]

        evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[cutoff], ignore_users=users_not_in_group)

        for label, recommender in recommender_object_dict.items():
            result_df, _ = evaluator_test.evaluateRecommender(recommender)
            if label in MAP_recommender_per_group:
                MAP_recommender_per_group[label].append(result_df.loc[cutoff]["MAP"])
            else:
                MAP_recommender_per_group[label] = [result_df.loc[cutoff]["MAP"]]
    else:
        print(f"Group {group_id} has no users.")

Group 0, #users in group 1227, average p.len 0.83, median 1.0, min 0, max 1
EvaluatorHoldout: Ignoring 2203 (17.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 11411 Users
EvaluatorHoldout: Processed 452 (100.0%) in 0.24 sec. Users per second: 1887
EvaluatorHoldout: Processed 452 (100.0%) in 0.25 sec. Users per second: 1839
EvaluatorHoldout: Processed 452 (100.0%) in 0.22 sec. Users per second: 2014
EvaluatorHoldout: Processed 452 (100.0%) in 0.19 sec. Users per second: 2337
EvaluatorHoldout: Processed 452 (100.0%) in 0.19 sec. Users per second: 2376
EvaluatorHoldout: Processed 452 (100.0%) in 0.20 sec. Users per second: 2278
EvaluatorHoldout: Processed 452 (100.0%) in 0.19 sec. Users per second: 2344
EvaluatorHoldout: Processed 452 (100.0%) in 0.27 sec. Users per second: 1689
EvaluatorHoldout: Processed 452 (100.0%) in 0.55 sec. Users per second: 823
EvaluatorHoldout: Processed 452 (100.0%) in 0.24 sec. Users per second: 1863
EvaluatorHoldout: Processed 45

In [39]:
import pandas as pd
pd.DataFrame(MAP_recommender_per_group)

,TopPop,UserKNNCF,ItemKNNCF,P3alpha,RP3,SLIM_ELASTIC,EASE_R,IALS,MULTVAE,hybrid,hybrid_new,KNN_H,superH,lowInt,hybrid_hi,h_hi_var,h_veryhi
0,0.004008,0.005147,0.004164,0.004142,0.004273,0.004256,0.004066,0.003902,0.005650,0.004304,0.004252,0.005332,0.004455,0.006384,0.004278,0.004278,0.004260
1,0.003806,0.008575,0.009482,0.007990,0.008850,0.007413,0.007257,0.009123,0.009305,0.008960,0.008524,0.008506,0.008989,0.008740,0.008481,0.008403,0.008500
2,0.002883,0.007786,0.010194,0.008885,0.009720,0.009344,0.008580,0.006886,0.005878,0.009710,0.009385,0.007811,0.009817,0.007850,0.009681,0.009173,0.009181
3,0.003605,0.011345,0.013497,0.012909,0.013071,0.012011,0.012315,0.011606,0.011037,0.012590,0.012519,0.012695,0.013289,0.013256,0.012220,0.012437,0.012437
4,0.004917,0.014229,0.019563,0.019374,0.019788,0.020198,0.018874,0.016451,0.016209,0.020664,0.020321,0.017764,0.020920,0.018093,0.020309,0.020261,0.020205
5,0.005840,0.020926,0.024120,0.025263,0.026338,0.026410,0.025866,0.022602,0.021708,0.027198,0.027027,0.023747,0.027613,0.024600,0.027011,0.027208,0.027019
6,0.007074,0.023616,0.027288,0.029371,0.030145,0.030401,0.028397,0.024915,0.023353,0.030560,0.030976,0.027065,0.031268,0.027946,0.031038,0.031054,0.031058
7,0.011057,0.028423,0.032071,0.033978,0.034341,0.033636,0.033483,0.027055,0.026732,0.034385,0.034956,0.032017,0.035504,0.032910,0.034977,0.034490,0.034540
8,0.009368,0.032887,0.038827,0.042475,0.042785,0.044987,0.042959,0.036124,0.032601,0.045129,0.045188,0.039466,0.045306,0.040403,0.045554,0.045590,0.045496
9,0.012196,0.037628,0.049639,0.050817,0.052923,0.056539,0.052525,0.040732,0.035012,0.056975,0.057339,0.047418,0.057194,0.047736,0.057376,0.057358,0.057545


In [38]:
#                         0, 1, 2, 3, 4,  5,  6,   7,   8,    9
#profile_length_cutoffs = [2, 4, 6, 8, 25, 50, 80, 100, 250, 1500]
df = pd.DataFrame(MAP_recommender_per_group)
for index, row in df.iterrows():
    sorted_row = row.sort_values(ascending=False)
    max_value = sorted_row.iloc[0]
    max_index = sorted_row.index[0]
    second_max_value = sorted_row.iloc[1]
    second_max_index = sorted_row.index[1]

    # Select users whose profile length is less than the cutoff for the group and greater than or equal to the cutoff for the previous group
    if index == 0:
        users_in_group_flag = np.logical_and(profile_length > 0, profile_length < profile_length_cutoffs[index])
    else:
        users_in_group_flag = np.logical_and(profile_length >= profile_length_cutoffs[index - 1], profile_length < profile_length_cutoffs[index])
    users_in_group = np.arange(len(profile_length))[users_in_group_flag]

    # Calculate the minimum and maximum profile lengths for the users in the group
    min_profile_length = profile_length[users_in_group_flag].min()
    max_profile_length = profile_length[users_in_group_flag].max()

    # Calculate the number of users in the group
    num_users_in_group = len(users_in_group)

    print(f"Gr {index}: Max val {max_value} at {max_index}, Second max val {second_max_value} at {second_max_index}, Min pl is {min_profile_length}, Max pl is {max_profile_length}, N users in group is {num_users_in_group}, diff {max_value - df['hybrid_hi'][index]}, improvement: {(max_value - df['hybrid_hi'][index])*num_users_in_group}")

Gr 0: Max val 0.0063844992274195815 at lowInt, Second max val 0.005650196656833826 at MULTVAE, Min pl is 1, Max pl is 1, N users in group is 1022, diff 0.0021063351594325028, improvement: 2.1526745329400176
Gr 1: Max val 0.009482085438677077 at ItemKNNCF, Second max val 0.009305491757260247 at MULTVAE, Min pl is 2, Max pl is 2, N users in group is 748, diff 0.001000612463634967, improvement: 0.7484581227989553
Gr 2: Max val 0.010193670759708496 at ItemKNNCF, Second max val 0.009817435359888186 at superH, Min pl is 3, Max pl is 3, N users in group is 608, diff 0.0005131276829390078, improvement: 0.31198163122691674
Gr 3: Max val 0.013497429370070877 at ItemKNNCF, Second max val 0.013289158430667863 at superH, Min pl is 4, Max pl is 4, N users in group is 490, diff 0.0012778276929220293, improvement: 0.6261355695317944
Gr 4: Max val 0.020920170283303233 at superH, Second max val 0.020663834890496353 at hybrid, Min pl is 5, Max pl is 8, N users in group is 1628, diff 0.000610872513562382,